# 03 - Transfer Learning with PyTorch (ResNet18 + CIFAR-10 subset)


In [ ]:
# !pip install -q torch torchvision matplotlib seaborn tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

os.makedirs("../figures", exist_ok=True)
os.makedirs("../models", exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", device)


In [ ]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)
classes = trainset.classes
print("Classes:", classes)


In [ ]:
resnet = models.resnet18(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = False

num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)
resnet = resnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)


In [ ]:
resnet.train()
epochs = 3
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in tqdm(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - loss: {running_loss/len(trainloader):.4f}")
print("Training done")


In [ ]:
resnet.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, cmap='Blues')
plt.title("Confusion Matrix - ResNet18 on CIFAR-10")
plt.savefig("../figures/pytorch_confusion_matrix.png", dpi=150)
plt.show()

torch.save(resnet.state_dict(), "../models/resnet18_cifar10.pth")
print("Saved PyTorch model.")


Notes:
- We froze the backbone and trained the head for speed.
- Record final accuracy (print or compute) for report.
